In [1]:
# !pip install pyngrok


In [2]:
# !pip install nltk scikit-learn mlflow xgboost


In [2]:
# # # Ensure ngrok is killed and reset
# !pkill ngrok
# !pip install pycaret

In [1]:
import nltk
nltk.download('popular')


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [2]:
import pandas as pd
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, make_scorer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import mlflow
import mlflow.sklearn
import os
from pyngrok import ngrok

# Load data
train_data = pd.read_csv("/content/train.csv")
test_data = pd.read_csv("/content/test.csv")


In [5]:
# Import the necessary libraries
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords as nltk_stopwords
import pandas as pd
import re
import spacy
import nltk
nlp = spacy.load("en_core_web_sm")

# Combine French stopwords with NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')
stpwords = ["amp", "https", "one", "new", "go", "see","say","know","come","think","make","want","new","via","s","u","news","rt"]
stopwords = stpwords + nltk_stopwords.words('english')

def clean_text(text):
    text = re.sub(r"\b(?:[a-zA-Z])\'\b", '', text)
    # Remove special characters at the beginning and end of words
    text = re.sub(r'\b[^\w\s]+\b', '', text)
    # Remove special characters at the beginning and end of words again to cover cases like "#word"
    text = re.sub(r'\b[^\w\s]+|[^\w\s]+\b', '', text)
    # Tokenization (dividing the text into words)
    words = nltk.word_tokenize(text.lower())  # Convert to lowercase
    # Remove punctuation, stopwords, and non-alphabetic words
    cleaned_words = [word for word in words if word not in stopwords and word.isalpha()]
    # Lemmatization
    doc = nlp(" ".join(cleaned_words))
    lemmatized_words = [token.lemma_ for token in doc]
    lemmatized_words = [word for word in lemmatized_words if word not in stopwords]
    # Join the lemmatized words to form the final text
    cleaned_text = " ".join(lemmatized_words)
    return cleaned_text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

# Concatenate 'text' and 'Keywords' if 'Keywords' is not NaN
df_train['text'] = df_train.apply(
    lambda row: f"{row['text']} {row['keyword']} " if pd.notna(row['keyword']) else row['text'],
    axis=1
)
# Apply preprocessing to the text data
df_train['cleaned_text'] = df_train ['text'].apply(clean_text)  # Remplacez 'text_column' par le nom de votre colonne de texte


In [7]:
import shutil
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer

warnings.filterwarnings("ignore")

# Set up MLflow tracking
local_registry = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(local_registry)

# Create an experiment
exp_name = "disaster_tweets"
experiment = mlflow.get_experiment_by_name(exp_name)
if not experiment:
    experiment_id = mlflow.create_experiment(exp_name)
else:
    experiment_id = experiment.experiment_id

In [8]:
# Define a function to train and evaluate models
def train_and_evaluate(model, X_train, y_train, X_val, y_val, model_name):
    with mlflow.start_run(run_name=model_name, experiment_id=experiment_id) as run:
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        # Calculate metrics for training set
        train_f1 = f1_score(y_train, y_train_pred, average='weighted')
        train_precision = precision_score(y_train, y_train_pred, average='weighted')
        train_recall = recall_score(y_train, y_train_pred, average='weighted')

        # Calculate metrics for validation set
        val_f1 = f1_score(y_val, y_val_pred, average='weighted')
        val_precision = precision_score(y_val, y_val_pred, average='weighted')
        val_recall = recall_score(y_val, y_val_pred, average='weighted')

        # Log parameters and metrics
        mlflow.log_params(model.get_params())
        mlflow.log_metric("train_f1_score", train_f1)
        mlflow.log_metric("train_precision", train_precision)
        mlflow.log_metric("train_recall", train_recall)
        mlflow.log_metric("val_f1_score", val_f1)
        mlflow.log_metric("val_precision", val_precision)
        mlflow.log_metric("val_recall", val_recall)

        # Log the model
        mlflow.sklearn.log_model(model, model_name)

        # Print metrics
        print(f'{model_name} Training F1 Score: {train_f1}')
        print(f'{model_name} Training Precision: {train_precision}')
        print(f'{model_name} Training Recall: {train_recall}')
        print(f'{model_name} Validation F1 Score: {val_f1}')
        print(f'{model_name} Validation Precision: {val_precision}')
        print(f'{model_name} Validation Recall: {val_recall}')

In [18]:
# # Load and preprocess data
# # Assuming `train_data` is a DataFrame with 'cleaned_text' and 'target'
# X = df_train['cleaned_text']
# y = df_train['target']

# # Vectorize the text data
# tfidf = TfidfVectorizer(max_features=5000)
# X_vect = tfidf.fit_transform(X)

# # Split the data into train and validation sets
# X_train_vect, X_val_vect, y_train, y_val = train_test_split(X_vect, y, test_size=0.2, random_state=50)

In [34]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split

# # Assuming `df_train` is a DataFrame with 'cleaned_text' and 'target'
# X = df_train['cleaned_text']
# y = df_train['target']

# # Vectorize the text data using Bag of Words
# count_vect = CountVectorizer(max_features=20000)
# X_vect = count_vect.fit_transform(X)

# # Split the data into train and validation sets
# X_train_vect, X_val_vect, y_train, y_val = train_test_split(X_vect, y, test_size=0.2, random_state=50)

# # Print shapes of the resulting datasets
# print(f"Training features shape: {X_train_vect.shape}")
# print(f"Validation features shape: {X_val_vect.shape}")
# print(f"Training labels shape: {y_train.shape}")
# print(f"Validation labels shape: {y_val.shape}")


Training features shape: (6090, 14671)
Validation features shape: (1523, 14671)
Training labels shape: (6090,)
Validation labels shape: (1523,)


In [48]:
# import pandas as pd
# import numpy as np
# from gensim.models import Word2Vec
# from sklearn.model_selection import train_test_split
# from nltk.tokenize import word_tokenize
# import nltk

# # Download necessary NLTK data
# nltk.download('punkt')

# # Assuming `df_train` is a DataFrame with 'cleaned_text' and 'target'
# X = df_train['cleaned_text']
# y = df_train['target']

# # Preprocess the text data
# def preprocess_text(text):
#     # Tokenize the text
#     tokens = word_tokenize(text.lower())
#     return tokens

# # Apply preprocessing to the text data
# X_tokenized = X.apply(preprocess_text)

# # Train Word2Vec model
# word2vec_model = Word2Vec(sentences=X_tokenized, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# # Transform text data into vectors by averaging the word vectors
# def transform_text_to_vector(tokens, model):
#     word_vectors = [model.wv[word] for word in tokens if word in model.wv]
#     if len(word_vectors) == 0:
#         return np.zeros(model.vector_size)
#     return np.mean(word_vectors, axis=0)

# X_vect = X_tokenized.apply(lambda tokens: transform_text_to_vector(tokens, word2vec_model))
# X_vect = np.vstack(X_vect)

# # Split the data into train and validation sets
# X_train_vect, X_val_vect, y_train, y_val = train_test_split(X_vect, y, test_size=0.2, random_state=50)

# # Print shapes of the resulting datasets
# print(f"Training features shape: {X_train_vect.shape}")
# print(f"Validation features shape: {X_val_vect.shape}")
# print(f"Training labels shape: {y_train.shape}")
# print(f"Validation labels shape: {y_val.shape}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training features shape: (6090, 100)
Validation features shape: (1523, 100)
Training labels shape: (6090,)
Validation labels shape: (1523,)


In [51]:
!pip install transformers


In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
from sklearn.model_selection import train_test_split

# Download necessary NLTK data
nltk.download('punkt')

# Load pre-trained GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Assuming you have downloaded `glove.6B.100d.txt`
glove_file_path = 'glove.6B.100d.txt'
embeddings_index = load_glove_embeddings(glove_file_path)

# Assuming `df_train` is a DataFrame with 'cleaned_text' and 'target'
X = df_train['cleaned_text']
y = df_train['target']

# Preprocess the text data
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    return tokens

# Apply preprocessing to the text data
X_tokenized = X.apply(preprocess_text)

# Transform text data into vectors by averaging the word vectors from GloVe
def transform_text_to_vector(tokens, embeddings_index, vector_size=100):
    word_vectors = [embeddings_index[word] for word in tokens if word in embeddings_index]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)

vector_size = 100
X_vect = X_tokenized.apply(lambda tokens: transform_text_to_vector(tokens, embeddings_index, vector_size))
X_vect = np.vstack(X_vect)

# Split the data into train and validation sets
X_train_vect, X_val_vect, y_train, y_val = train_test_split(X_vect, y, test_size=0.2, random_state=50)

# Print shapes of the resulting datasets
print(f"Training features shape: {X_train_vect.shape}")
print(f"Validation features shape: {X_val_vect.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation labels shape: {y_val.shape}")

# Now you can use the vectorized data for training your models


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'

In [49]:
# Define a fixed random seed
random_seed = 50

# Define classification models with fixed random seed
models = {
    'RandomForestClassifier': RandomForestClassifier(n_estimators=10, random_state=random_seed),
    'SVC': SVC(random_state=random_seed),  # SVC does not use random_state directly, but the seed is useful for reproducibility in some cases
    # 'Naive Bayes': MultinomialNB(),  # Naive Bayes does not use random_state
    'KNN': KNeighborsClassifier(),  # KNN does not use random_state
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=random_seed)
}

In [50]:
# Train and evaluate each model
for model_name, model in models.items():
    train_and_evaluate(model, X_train_vect, y_train, X_val_vect, y_val, model_name)

RandomForestClassifier Training F1 Score: 0.9759672410527488
RandomForestClassifier Training Precision: 0.9763477147866467
RandomForestClassifier Training Recall: 0.9760262725779967
RandomForestClassifier Validation F1 Score: 0.7007591090760764
RandomForestClassifier Validation Precision: 0.7060603458924084
RandomForestClassifier Validation Recall: 0.7058437294812869
SVC Training F1 Score: 0.6982914699818777
SVC Training Precision: 0.7355257740257387
SVC Training Recall: 0.7172413793103448
SVC Validation F1 Score: 0.6844619800028393
SVC Validation Precision: 0.7203298914938522
SVC Validation Recall: 0.7019041365725541
KNN Training F1 Score: 0.8010795509773824
KNN Training Precision: 0.8050033692290346
KNN Training Recall: 0.8037766830870279
KNN Validation F1 Score: 0.7012320883017586
KNN Validation Precision: 0.703442959597572
KNN Validation Recall: 0.7045305318450427
XGBoost Training F1 Score: 0.9899756165629713
XGBoost Training Precision: 0.9900202384987495
XGBoost Training Recall: 0

In [ ]:
# Start MLflow UI in the background
get_ipython().system_raw("mlflow ui --backend-store-uri sqlite:///mlruns.db --port 5000 &")

In [ ]:
NGROK_AUTH_TOKEN = "2iY5u5ACBp6ivPRy41sjmzYrg0t_2Y8h5Gs7Tfbi4AHW36Hux"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000)
print("MLflow Tracking UI:", public_url)

# Transition models to production
client = MlflowClient()
for model_name in models.keys():
    try:
        model_versions = client.search_model_versions(f"name='{model_name}'")
        if model_versions:
            # Get the latest model version
            latest_version = model_versions[-1]
            client.transition_model_version_stage(
                name=model_name,
                version=latest_version.version,
                stage="production"
            )
    except Exception as e:
        print(f"Error transitioning model {model_name}: {e}")


MLflow Tracking UI: NgrokTunnel: "https://6e83-35-236-227-176.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
import pandas as pd
from pycaret.regression import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Assuming `train_data` is a DataFrame with 'cleaned_text' and 'target'
X = train_data['cleaned_text']
y = train_data['target']

# Vectorize the text data
tfidf = TfidfVectorizer(max_features=5000)
X_vect = tfidf.fit_transform(X)

# Convert the vectorized data to a DataFrame
X_vect_df = pd.DataFrame(X_vect.toarray(), columns=tfidf.get_feature_names_out())

# Add the target column to the DataFrame
X_vect_df['target'] = y.values

# Initialize PyCaret's regression environment
regression_setup = setup(data=X_vect_df, target='target', session_id=123)

# Compare models
best_model = compare_models()

# Pull the results
results = pull()

# Print the results
print(results)

# Finalize the best model (optional)
final_model = finalize_model(best_model)

# Print the best model details
print(final_model)


,Description,Value
0,Session id,123
1,Target,target
2,Target type,Regression
3,Original data shape,"(7613, 5000)"
4,Transformed data shape,"(7613, 5000)"
5,Transformed train set shape,"(5329, 5000)"
6,Transformed test set shape,"(2284, 5000)"
7,Numeric features,4999
8,Preprocess,True
9,Imputation type,simple


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:07:27
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,0.3214,0.1552,0.3937,0.3662,0.2770,0.3719,56.1970
ridge,Ridge Regression,0.3196,0.1564,0.3953,0.3612,0.2778,0.3695,5.2540
lar,Least Angle Regression,0.3484,0.1628,0.4034,0.3350,0.2827,0.4152,7.2010
rf,Random Forest Regressor,0.3233,0.1786,0.4223,0.2706,0.2986,0.3664,400.3790
omp,Orthogonal Matching Pursuit,0.3329,0.1834,0.4281,0.2507,0.2911,0.4070,7.4830
huber,Huber Regressor,0.3429,0.2147,0.4630,0.1230,0.3093,0.3942,72.0410
knn,K Neighbors Regressor,0.4299,0.2192,0.4677,0.1037,0.3328,0.4520,3.6700
lasso,Lasso Regression,0.4908,0.2454,0.4954,-0.0026,0.3486,0.5684,2.4810
en,Elastic Net,0.4908,0.2454,0.4954,-0.0026,0.3486,0.5684,2.4880
llar,Lasso Least Angle Regression,0.4908,0.2454,0.4954,-0.0026,0.3486,0.5684,2.4160


Processing:   0%|          | 0/81 [00:00<?, ?it/s]